# Project Work Part 1

#### a)
Download the data and display it

In [1]:
import numpy as np
import polars as pl

# read data
links_df = pl.read_csv("../data/links.csv", schema={"movieId": pl.Int32, "imdb": pl.Int32, "tmdbId": pl.Int32})
movies_df = (
    pl.read_csv("../data/movies.csv", schema={"movieId": pl.Int32, "title": pl.String, "genres": pl.String})
    .with_row_index()
    .with_columns(
        pl.col("index")
        .cast(pl.Int32)
    )
)
ratings_df = pl.read_csv("../data/ratings.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "rating": pl.Float32, "timestamp": pl.Int32})
tags_df = pl.read_csv("../data/tags.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "tag": pl.String, "timestamp": pl.Int32})

print("Number of ratings: ", ratings_df.height)

Number of ratings:  100836


In [2]:
print(links_df)

shape: (9_742, 3)
┌─────────┬─────────┬────────┐
│ movieId ┆ imdb    ┆ tmdbId │
│ ---     ┆ ---     ┆ ---    │
│ i32     ┆ i32     ┆ i32    │
╞═════════╪═════════╪════════╡
│ 1       ┆ 114709  ┆ 862    │
│ 2       ┆ 113497  ┆ 8844   │
│ 3       ┆ 113228  ┆ 15602  │
│ 4       ┆ 114885  ┆ 31357  │
│ 5       ┆ 113041  ┆ 11862  │
│ …       ┆ …       ┆ …      │
│ 193581  ┆ 5476944 ┆ 432131 │
│ 193583  ┆ 5914996 ┆ 445030 │
│ 193585  ┆ 6397426 ┆ 479308 │
│ 193587  ┆ 8391976 ┆ 483455 │
│ 193609  ┆ 101726  ┆ 37891  │
└─────────┴─────────┴────────┘


In [3]:
print(movies_df)

shape: (9_742, 4)
┌───────┬─────────┬─────────────────────────────────┬─────────────────────────────────┐
│ index ┆ movieId ┆ title                           ┆ genres                          │
│ ---   ┆ ---     ┆ ---                             ┆ ---                             │
│ i32   ┆ i32     ┆ str                             ┆ str                             │
╞═══════╪═════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 0     ┆ 1       ┆ Toy Story (1995)                ┆ Adventure|Animation|Children|C… │
│ 1     ┆ 2       ┆ Jumanji (1995)                  ┆ Adventure|Children|Fantasy      │
│ 2     ┆ 3       ┆ Grumpier Old Men (1995)         ┆ Comedy|Romance                  │
│ 3     ┆ 4       ┆ Waiting to Exhale (1995)        ┆ Comedy|Drama|Romance            │
│ 4     ┆ 5       ┆ Father of the Bride Part II (1… ┆ Comedy                          │
│ …     ┆ …       ┆ …                               ┆ …                               │
│ 9737  ┆ 1935

In [4]:
print(ratings_df)

shape: (100_836, 4)
┌────────┬─────────┬────────┬────────────┐
│ userId ┆ movieId ┆ rating ┆ timestamp  │
│ ---    ┆ ---     ┆ ---    ┆ ---        │
│ i32    ┆ i32     ┆ f32    ┆ i32        │
╞════════╪═════════╪════════╪════════════╡
│ 1      ┆ 1       ┆ 4.0    ┆ 964982703  │
│ 1      ┆ 3       ┆ 4.0    ┆ 964981247  │
│ 1      ┆ 6       ┆ 4.0    ┆ 964982224  │
│ 1      ┆ 47      ┆ 5.0    ┆ 964983815  │
│ 1      ┆ 50      ┆ 5.0    ┆ 964982931  │
│ …      ┆ …       ┆ …      ┆ …          │
│ 610    ┆ 166534  ┆ 4.0    ┆ 1493848402 │
│ 610    ┆ 168248  ┆ 5.0    ┆ 1493850091 │
│ 610    ┆ 168250  ┆ 5.0    ┆ 1494273047 │
│ 610    ┆ 168252  ┆ 5.0    ┆ 1493846352 │
│ 610    ┆ 170875  ┆ 3.0    ┆ 1493846415 │
└────────┴─────────┴────────┴────────────┘


In [5]:
print(tags_df)

shape: (3_683, 4)
┌────────┬─────────┬──────────────────┬────────────┐
│ userId ┆ movieId ┆ tag              ┆ timestamp  │
│ ---    ┆ ---     ┆ ---              ┆ ---        │
│ i32    ┆ i32     ┆ str              ┆ i32        │
╞════════╪═════════╪══════════════════╪════════════╡
│ 2      ┆ 60756   ┆ funny            ┆ 1445714994 │
│ 2      ┆ 60756   ┆ Highly quotable  ┆ 1445714996 │
│ 2      ┆ 60756   ┆ will ferrell     ┆ 1445714992 │
│ 2      ┆ 89774   ┆ Boxing story     ┆ 1445715207 │
│ 2      ┆ 89774   ┆ MMA              ┆ 1445715200 │
│ …      ┆ …       ┆ …                ┆ …          │
│ 606    ┆ 7382    ┆ for katie        ┆ 1171234019 │
│ 606    ┆ 7936    ┆ austere          ┆ 1173392334 │
│ 610    ┆ 3265    ┆ gun fu           ┆ 1493843984 │
│ 610    ┆ 3265    ┆ heroic bloodshed ┆ 1493843978 │
│ 610    ┆ 168248  ┆ Heroic Bloodshed ┆ 1493844270 │
└────────┴─────────┴──────────────────┴────────────┘


#### b)
User-based collaborative filtering using the Pearson correlation function.

The pearson correlation function is found in polars so there is no need to implement it from scratch. The function will take into account only the ratings that are rated by both users. However, pearson correlation function can produce nan values which must be filtered out later.

Let's calculate top similar users for userId 1 using Pearson correlation


In [6]:
def get_top_n_most_similar_users_pearson(user_id, n):
    distance_df = (
        ratings_df
        .drop("timestamp")
        .pivot("userId", index="movieId")
        .drop("movieId")
        .select(pl.corr(pl.all(), str(user_id), method="pearson"))
        .transpose(include_header=True, header_name="userId", column_names=["correlation"])
        .cast({"userId": pl.Int16})
        .filter(pl.col("correlation").is_not_nan()) # filter out NaN values
        .sort(["correlation", "userId"], descending=[True, False])
        .filter(pl.col("userId") != user_id) # filter out the user itself
        .filter(pl.col("correlation") > 0.2) # filter out users with low correlation
        .limit(n)
    )
    return (
        distance_df
        .select("userId")
        .to_numpy()
        .reshape(-1)
    ), (
        distance_df
        .select("correlation")
        .to_numpy()
        .reshape(-1)
    )

# Calculate the top 50 most similar users to userId 1
user_ids, similarity_scores = get_top_n_most_similar_users_pearson(1, 50)
print(user_ids)
print(similarity_scores)

[106 146 333 550 598 473 511   9  13 366 401 535 154 481  90 499 157 139
 476 487 210 114 530  49 162 297 207  44 430 394 574 421 248 173  65 505
 369 435 375 431 351  72 206 120 344 112  99 445 494 467]
[1.         1.         1.         1.         1.         0.9622505
 0.9258201  0.91855866 0.8783101  0.8728716  0.8669214  0.86640024
 0.8660254  0.8660254  0.8215838  0.8029551  0.80178374 0.7905694
 0.78693587 0.7745967  0.7676495  0.7592566  0.75592893 0.75
 0.7083333  0.7062815  0.7016464  0.6844476  0.65465367 0.6506001
 0.6437963  0.64122343 0.62469506 0.6172134  0.61237246 0.61237246
 0.61209774 0.6109598  0.6061281  0.6030227  0.6        0.5964321
 0.5962848  0.59487444 0.58760905 0.58389246 0.58345    0.56906945
 0.5673498  0.5669467 ]


Pearson correlation function has two major problems. If the standard deviation of either of the ratings is 0 the function will give nan result as there is a division by zero. This is a problem since in a case where the user has given same score for all movies the standard deviation will be 0 and the function will not work. For example users with ratings [5, 5, 5, 5] and [5, 5, 5, 5] will have a correlation of nan even though they are exactly similar. Also users with low and high ratings e.g. [5, 4, 5, 5] and [2, 1, 2, 2] will still have high Pearson correlation even though they are not similar at all.

In [7]:
# Example of the problems with Pearson correlation
df = pl.DataFrame(
    {
    "a": [5, 5, 5, 5],
    "b": [5, 5, 5, 5],
    "c": [5, 4, 5, 5],
    "d": [2, 1, 2, 2],
    }
)

print(df.select(pl.corr("a", "b", method="pearson")))
print(df.select(pl.corr("c", "d", method="pearson")))

shape: (1, 1)
┌─────┐
│ a   │
│ --- │
│ f64 │
╞═════╡
│ NaN │
└─────┘
shape: (1, 1)
┌─────┐
│ c   │
│ --- │
│ f64 │
╞═════╡
│ 1.0 │
└─────┘


#### c)
Prediction function from course slides

In [8]:
def predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, num_of_movies):
    """
    The prediction function from course slides.

    :param user_ratings_mean: mean of user ratings
    :param similar_users_ratings: sparse numpy 2D matrix of similar users ratings. Each row is a user and each column is a movie
    :param similar_users_ratings_mean: numpy array of mean ratings of the similar users
    :param similarity_scores: numpy array of similarity scores between the user and the similar users
    :param candidate_movie_indices: numpy array of movie indices that the similar users have rated
    :param num_of_movies: number of movies to recommend
    :return: top ranked candidate movie indices, and their predicted ratings
    """
    
    # Compute ratings only for the candidate movies to save computation
    candidate_similar_users_ratings = np.take(similar_users_ratings, candidate_movie_indices, axis=1)

    # Subtract mean rating from each rating, (r_b,p - r_b)
    delta_ratings = candidate_similar_users_ratings - similar_users_ratings_mean.reshape(similar_users_ratings_mean.shape[0], 1)

    # Sum sim(a,b) * (r_b,p - r_b)
    numerator = np.sum(similarity_scores.reshape(-1, 1) * delta_ratings, axis=0)

    # Sum |sim(a,b)|, taking absolute value to avoid negative bias even though it should not be possible
    denominator = np.sum(np.abs(similarity_scores))

    # Sum sim(a,b) * (r_b,p - r_b) / Sum |sim(a,b)|
    bias = numerator / denominator

    # pred(a,p) = r_a + bias
    predicted_ratings = user_ratings_mean + bias

    # Rank the movies by predicted rating
    indices = np.flip(np.argsort(predicted_ratings))
    predicted_movie_indices = candidate_movie_indices[indices][:num_of_movies]
    sorted_ratings = predicted_ratings[indices][:num_of_movies]

    return predicted_movie_indices, sorted_ratings

Let's predict movies for userId 1 using the top 50 most similar users and the prediction function from the course slides

In [9]:
# Define some helper functions to extract data from the dataframes

# ratings
def get_ratings_for_users(user_ids):
    return (
        ratings_df
        .drop("timestamp")
        .sort("movieId")
        .pivot("userId", index="movieId")
        .drop("movieId")
        .select([str(user_id) for user_id in user_ids])
        .transpose(include_header=True, header_name="userId", column_names=user_ids)
        .drop("userId")
        .to_numpy()
    )
user_ids = np.array([1, 2, 3, 4, 5])
print(get_ratings_for_users(user_ids))

# ratings mean
def get_mean_ratings_for_users(user_ids):
    return (
        ratings_df
        .drop("timestamp")
        .sort("movieId")
        .pivot("userId", index="movieId")
        .drop("movieId")
        .select([str(user_id) for user_id in user_ids.tolist()])
        .select(pl.all().mean())
        .transpose(include_header=True, header_name="userId", column_names=["ratings_mean"])
        .cast({"userId": pl.Int16})
        .filter(pl.col("userId").is_in(user_ids))
        .select("ratings_mean")
        .to_numpy()
        .reshape(-1)
    )
print(get_mean_ratings_for_users(user_ids))

# movie indices for movie ids
def get_movie_indices_for_movie_ids(movie_ids):
    return (
        ratings_df
        .select("movieId")
        .unique()
        .sort("movieId")
        .with_row_index()
        .filter(pl.col("movieId").is_in(movie_ids))
        .select("index")
        .to_numpy()
        .reshape(-1)
    )

print(get_movie_indices_for_movie_ids(np.array([1, 2, 3, 100, 112])))

# single user rating mean
def get_user_rating_mean(user_id):
    return get_mean_ratings_for_users(np.array([user_id]))[0]

# candidate movie ids
def get_candidate_movie_ids(user_id, similar_user_ids):
    rated_by_user = (
        ratings_df
        .filter(pl.col("userId") == user_id)
        .select("movieId")
        .to_numpy()
        .reshape(-1)
    )

    return (
        ratings_df
        .filter(
            pl.col("userId").is_in(similar_user_ids), # only include ratings from similar users
            pl.col("movieId").is_in(rated_by_user).not_() # only include movies that the user has not rated
        )
        .group_by("movieId")
        .agg()
        .select("movieId")
        .to_numpy()
        .reshape(-1)
    )

print(get_candidate_movie_ids(1, np.array([2, 3, 4, 5, 6]))[:10])

[[ 4. nan  4. ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [ 4. nan nan ... nan nan nan]]
[4.3663793 3.9482758 2.4358974 3.5555556 3.6363637]
[ 0  1  2 88 99]
[  837   468 74458  1093   775   640   102   377   416   575]


Now predict 10 movies for userId 1 using the top 50 most similar users

In [10]:
# Recommend 10 movies for userId 1
user_id = 1

user_ratings_mean = get_user_rating_mean(user_id)
top_50_most_similar_user_ids, similarity_scores = get_top_n_most_similar_users_pearson(1, 50)
similar_users_ratings = np.nan_to_num(get_ratings_for_users(top_50_most_similar_user_ids))
similar_users_ratings_mean = get_mean_ratings_for_users(top_50_most_similar_user_ids)
candidate_movie_ids = get_candidate_movie_ids(user_id, top_50_most_similar_user_ids)
candidate_movie_indices = get_movie_indices_for_movie_ids(candidate_movie_ids)

predicted_movie_indices, ratings = predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, 10)

print('predicted movie indices:')
print(predicted_movie_indices)
print()
print('predicted ratings:')
print(ratings)
print()

top_10_ratings_df = (
    pl.DataFrame({"index": predicted_movie_indices,
                  "predicted_rating": ratings})
    .with_columns(pl.col("index").cast(pl.Int32))
)

# print top 10 movies
top_10_movies_df = (
    movies_df
    .filter(pl.col("index").is_in(predicted_movie_indices))
    .join(top_10_ratings_df, on="index")
    .drop("index")
)

print(top_10_movies_df)

predicted movie indices:
[ 277 7355 3633 4791 8358 4131 6755 6693  123 4421]

predicted ratings:
[2.3414884 1.8985755 1.8432145 1.7272666 1.5624275 1.5383899 1.5101435
 1.5003035 1.4971073 1.4793124]

shape: (10, 4)
┌─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────────┐
│ movieId ┆ title                           ┆ genres                          ┆ predicted_rating │
│ ---     ┆ ---                             ┆ ---                             ┆ ---              │
│ i32     ┆ str                             ┆ str                             ┆ f32              │
╞═════════╪═════════════════════════════════╪═════════════════════════════════╪══════════════════╡
│ 318     ┆ Shawshank Redemption, The (199… ┆ Crime|Drama                     ┆ 2.341488         │
│ 78499   ┆ Toy Story 3 (2010)              ┆ Adventure|Animation|Children|C… ┆ 1.898576         │
│ 4988    ┆ White Water Summer (1987)       ┆ Adventure                       ┆ 1.843215   

From the results we can see that the Pearson correlation function did not work quite well as a user similarity measure. The best predicted rating is only 2.34 which is considered a bad rating.

#### d)

New similarity function

We chose euclidean distance as a distance measure for the basis of the new similarity function since it is very accurate measure of distance. Euclidean distance does not work well on sparse arrays but it is not a problem since we are only interested in common predicted movies between two users in order to calculate their similarity. We use nan_eucliden_distance from sklearn which takes into account only commonly rated movies of two users. 

To convert euclidean distance into a similarity measure we do the following:
- Calculate average distance from user ratings for all users
- Do max scaling for the average distances in order to scale it to range [0-1]
- Inverse distance to make it a similarity score
- Users with less than 5 commonly rated movies with the queried user will be ignored since there are too few common ratings 

In [11]:
from sklearn.metrics.pairwise import nan_euclidean_distances


def get_top_n_most_similar_users_euclidean(user_id, n):
    rating_vectors_df = (
        ratings_df
        .drop("timestamp")
        .pivot("movieId", index="userId")
    )
    
    user_ratings = rating_vectors_df.filter(pl.col("userId") == user_id).drop("userId").to_numpy()
    
    rating_vectors = (
        rating_vectors_df
        .drop("userId")
        .to_numpy()
    )
    
    # Calculate distance
    distances = nan_euclidean_distances(rating_vectors, user_ratings).reshape(-1)
    
    # Find nan values and count the amount of common movies between user and all users
    nans_in_user_ratings = np.isnan(user_ratings[0])
    nans_in_rating_vectors = np.isnan(rating_vectors)
    common_rated_movies = 1 - np.logical_or(nans_in_user_ratings, nans_in_rating_vectors)
    common_rated_movies_count = np.sum(common_rated_movies, axis=1).astype(np.float32) 
    
    # Ignore users withe less than 5 common movies
    common_rated_movies_count[common_rated_movies_count < 5] = np.nan 
    
    # Count average distances
    average_distances = distances / common_rated_movies_count
    
    # Max scaling
    max_average_distance = np.nanmax(average_distances)
    
    # Distance to similarity. Nan values are set to zero
    similarities = np.nan_to_num(1 - (average_distances / max_average_distance))
    
    distance_df = (
        pl.DataFrame(
            {
                "userId": rating_vectors_df.select("userId"),
                "similarity": similarities
            }
        )
        .sort("similarity", descending=True)
        .limit(n)
    )
    
    return (
        distance_df
        .select("userId")
        .to_numpy()
        .reshape(-1)
    ), (
        distance_df
        .select("similarity")
        .to_numpy()
        .reshape(-1)
    )

# Calculate the top 50 most similar users to userId 1
user_ids, similarity_scores = get_top_n_most_similar_users_euclidean(1, 50)
print(user_ids)
print(similarity_scores)

[  1  77 414 599 448 274 380 474 288 597  64 590 480 249 555  45  91 182
 469 477 561  57 226  68 387 608 202 239 483  19 307 600 452 368 610 135
  18 438 217 606 580 357 318 219  66 177 313 603 122 266]
[1.         1.         0.99025923 0.98476946 0.98414505 0.9835183
 0.9833781  0.98232174 0.9821547  0.9821045  0.98187095 0.9818385
 0.9814875  0.9805435  0.9804517  0.98007464 0.9800217  0.9799595
 0.97988224 0.97982675 0.97919303 0.979186   0.97894776 0.9789022
 0.97859925 0.9784891  0.97696626 0.97677267 0.9766887  0.9758603
 0.9745885  0.97452873 0.97402376 0.9738219  0.97379315 0.9736019
 0.9730914  0.97290325 0.9720469  0.9720361  0.9718734  0.97155267
 0.971082   0.9710255  0.9706603  0.9704868  0.9703443  0.9702841
 0.970108   0.9698808 ]


Let's try our own similarity function to produce recommendations

In [119]:
# Recommend 10 movies for userId 1
user_id = 1

user_ratings_mean = get_user_rating_mean(user_id)
top_50_most_similar_user_ids, similarity_scores = get_top_n_most_similar_users_euclidean(1, 50)
similar_users_ratings = np.nan_to_num(get_ratings_for_users(top_50_most_similar_user_ids))
similar_users_ratings_mean = get_mean_ratings_for_users(top_50_most_similar_user_ids)
candidate_movie_ids = get_candidate_movie_ids(user_id, top_50_most_similar_user_ids)
candidate_movie_indices = get_movie_indices_for_movie_ids(candidate_movie_ids)

predicted_movie_indices, ratings = predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, 10)

print('predicted movie indices:')
print(predicted_movie_indices)
print()
print('predicted ratings:')
print(ratings)
print()

top_10_ratings_df = (
    pl.DataFrame({"index": predicted_movie_indices,
                  "predicted_rating": ratings})
    .with_columns(pl.col("index").cast(pl.Int32))
)

# print top 10 movies
top_10_movies_df = (
    movies_df
    .filter(pl.col("index").is_in(predicted_movie_indices))
    .join(top_10_ratings_df, on="index")
    .drop("index")
)

print(top_10_movies_df)

predicted movie indices:
[ 507 2077  659  793   31 2194  277  901 3633  921]

predicted ratings:
[4.5639324 4.363309  4.333353  4.303628  4.263337  4.205062  4.1032257
 3.9740975 3.967545  3.9452226]

shape: (10, 4)
┌─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────────┐
│ movieId ┆ title                           ┆ genres                          ┆ predicted_rating │
│ ---     ┆ ---                             ┆ ---                             ┆ ---              │
│ i32     ┆ str                             ┆ str                             ┆ f32              │
╞═════════╪═════════════════════════════════╪═════════════════════════════════╪══════════════════╡
│ 589     ┆ Terminator 2: Judgment Day (19… ┆ Action|Sci-Fi                   ┆ 4.563932         │
│ 2761    ┆ Iron Giant, The (1999)          ┆ Adventure|Animation|Children|D… ┆ 4.363309         │
│ 858     ┆ Godfather, The (1972)           ┆ Crime|Drama                     ┆ 4.333353   

From the results we can see how our own euclidean distance based similarity measure is much better than the pearson correlation based similarity measure. All the predicted ratings are over 3.5 which is great!

#### e)
User-based collaborative filtering for group recommendations.

Let's do group recommendations for a group of users using our own similarity function.

In [88]:
# Create a group of users by picking the first 5 userIds
user_ids = [1, 2, 3, 4, 5]

# Combine all the above functions to predict movies for a single user
def predict_movies_for_user(user_id, num_of_movies):
    similar_user_ids, similarity_scores = get_top_n_most_similar_users_euclidean(user_id, 50)
    
    user_ratings_mean = get_user_rating_mean(user_id)
    # print(user_ratings_mean)
    similar_users_ratings = np.nan_to_num(get_ratings_for_users(similar_user_ids))
    # print(similar_users_ratings)
    similar_users_ratings_mean = get_mean_ratings_for_users(similar_user_ids)
    # print(similar_users_ratings_mean)
    candidate_movie_ids = get_candidate_movie_ids(user_id, similar_user_ids)

    candidate_movie_indices = get_movie_indices_for_movie_ids(candidate_movie_ids)

    return predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, num_of_movies)


n_movies = 500
predicted_movie_indices = np.ndarray(shape=(len(user_ids), n_movies), dtype=np.int32)
predicted_ratings = np.ndarray(shape=(len(user_ids), n_movies), dtype=np.float32)

for index, user_id in enumerate(user_ids):
    user_movie_indices, ratings = predict_movies_for_user(user_id, n_movies)
    predicted_movie_indices[index] = user_movie_indices
    predicted_ratings[index] = ratings

# Print top 10 movies for each user
print('Predicted top 10 movie indices:')
print(predicted_movie_indices[:, :10])
print()
print('Predicted top 10 ratings:')
print(predicted_ratings[:, :10])
print()

# Combine the predictions into a dataframe
predictions_df = (
    pl.DataFrame({
         "movie_index": predicted_movie_indices.reshape(-1),
         "predicted_rating": predicted_ratings.reshape(-1),
         "userId": np.array([[user_id] * n_movies for user_id in user_ids]).reshape(-1)
    })
    .unique()
    )

# Find movies which have rating for all users in the group
movie_ratings_count = (
    predictions_df
    .group_by("movie_index")
    .agg(
        pl.count("predicted_rating")
        .alias("ratings_count")
    )
    .filter(pl.col("ratings_count") >= 5)
    .sort("ratings_count", descending=True)
)

# filter predictions_df to only include movies that have been rated by all users in the group
predictions_df = (
    predictions_df
    .filter(
        pl.col("movie_index")
        .is_in(movie_ratings_count.select("movie_index")
        )
    )
    .sort("movie_index")
    .pivot("movie_index", index="userId")
)

print("predicted ratings")
print(predictions_df)

Predicted top 10 movie indices:
[[ 507 2077  659  793   31 2194  277  901 3633  921]
 [2224 1938  314  257  510  224 4791  659 3633  897]
 [ 897  224  910  257  899 1938  510  914  901  314]
 [ 659  314  277  910   46  827  835  968  914  418]
 [ 314  510  418  334  436   43   31  505  134  615]]

Predicted top 10 ratings:
[[4.5639324 4.363309  4.333353  4.303628  4.263337  4.205062  4.1032257
  3.9740975 3.967545  3.9452226]
 [4.274893  4.1791797 4.073797  4.0625973 3.767443  3.516509  3.5092556
  3.481159  3.4163282 3.3739378]
 [3.161363  3.1214013 2.912777  2.8910916 2.805883  2.7887506 2.6909418
  2.6833398 2.5243783 2.5198698]
 [3.7588086 3.7539608 3.6430898 3.62391   3.4937088 3.3878906 3.384595
  3.3761842 3.3065298 3.3033566]
 [4.342993  3.7937462 3.6620479 3.2765875 3.1103246 3.066608  3.0409098
  2.652749  2.5919852 2.56535  ]]

predicted ratings
shape: (5, 156)
┌────────┬──────────┬──────────┬──────────┬───┬──────────┬──────────┬──────────┬──────────┐
│ userId ┆ 1        ┆ 9

Average aggregated recommendations

In [13]:
# Calculate the average rating for each movie
average_ratings_df = (
    predictions_df
    .drop("userId")
    .mean()
)

print("Average ratings:")
print(average_ratings_df)
print()

# find the top 10 movies based on the average rating
top_10_movies_avg_df = (
    average_ratings_df
    .transpose(include_header=True, header_name="index", column_names=["avg_rating"])
    .with_columns(
        pl.col("index")
        .cast(pl.Int32)
    )
    .sort("avg_rating", descending=True)
    .limit(10)
    .join(movies_df, on="index", how="left")
)

print("Top 10 movies:")
print(top_10_movies_avg_df)
print()

Average ratings:
shape: (1, 155)
┌──────────┬─────────┬──────────┬──────────┬───┬──────────┬──────────┬──────────┬──────────┐
│ 1        ┆ 9       ┆ 15       ┆ 92       ┆ … ┆ 6755     ┆ 7022     ┆ 7026     ┆ 7195     │
│ ---      ┆ ---     ┆ ---      ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ f32      ┆ f32     ┆ f32      ┆ f32      ┆   ┆ f32      ┆ f32      ┆ f32      ┆ f32      │
╞══════════╪═════════╪══════════╪══════════╪═══╪══════════╪══════════╪══════════╪══════════╡
│ 1.913214 ┆ 1.84762 ┆ 1.848369 ┆ 1.594528 ┆ … ┆ 1.671101 ┆ 1.646907 ┆ 1.561117 ┆ 1.562628 │
└──────────┴─────────┴──────────┴──────────┴───┴──────────┴──────────┴──────────┴──────────┘

Top 10 movies:
shape: (10, 5)
┌───────┬────────────┬─────────┬─────────────────────────────────┬──────────────────────────┐
│ index ┆ avg_rating ┆ movieId ┆ title                           ┆ genres                   │
│ ---   ┆ ---        ┆ ---     ┆ ---                             ┆ ---                      │
│ i

Least misery aggregated recommendations

In [14]:
# Calculate least misery rating for each movie
least_misery_ratings_df = (
    predictions_df
    .drop("userId")
    .min()
)

print("Least misery ratings:")
print(least_misery_ratings_df)
print()

# find the top 10 movies based on the least misery rating
top_10_movies_least_misery_df = (
    least_misery_ratings_df
    .transpose(include_header=True, header_name="index", column_names=["least_misery_rating"])
    .with_columns(
        pl.col("index")
        .cast(pl.Int32)
    )
    .sort("least_misery_rating", descending=True)
    .limit(10)
    .join(movies_df, on="index", how="left")
)

print("Top 10 movies:")
print(top_10_movies_least_misery_df)
print()

Least misery ratings:
shape: (1, 155)
┌──────────┬──────────┬──────────┬─────────┬───┬──────────┬──────────┬──────────┬─────────┐
│ 1        ┆ 9        ┆ 15       ┆ 92      ┆ … ┆ 6755     ┆ 7022     ┆ 7026     ┆ 7195    │
│ ---      ┆ ---      ┆ ---      ┆ ---     ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ f32      ┆ f32      ┆ f32      ┆ f32     ┆   ┆ f32      ┆ f32      ┆ f32      ┆ f32     │
╞══════════╪══════════╪══════════╪═════════╪═══╪══════════╪══════════╪══════════╪═════════╡
│ 0.978737 ┆ 1.145303 ┆ 0.973632 ┆ 0.50591 ┆ … ┆ 0.674368 ┆ 0.685756 ┆ 0.322654 ┆ 0.56039 │
└──────────┴──────────┴──────────┴─────────┴───┴──────────┴──────────┴──────────┴─────────┘

Top 10 movies:
shape: (10, 5)
┌───────┬─────────────────────┬─────────┬─────────────────────────────┬────────────────────────────┐
│ index ┆ least_misery_rating ┆ movieId ┆ title                       ┆ genres                     │
│ ---   ┆ ---                 ┆ ---     ┆ ---                         ┆ ---          

In [15]:
# Find movies that appear in both top 10 lists
common_movies = (
    top_10_movies_avg_df
    .join(top_10_movies_least_misery_df, on="index", how="inner")
    .select(["index", "title", "avg_rating", "least_misery_rating"])
)

print("Common movies:")
print(common_movies)

# Find movies that appear in only one of the top 10 lists
unique_movies_avg = (
    top_10_movies_avg_df
    .join(top_10_movies_least_misery_df, on="index", how="anti")
    .select(["index", "title", "avg_rating"])
)

print("Unique movies avg:")
print(unique_movies_avg)
print()

unique_movies_least_misery = (
    top_10_movies_least_misery_df
    .join(top_10_movies_avg_df, on="index", how="anti")
    .select(["index", "title", "least_misery_rating"])
)

print("Unique movies least misery:")
print(unique_movies_least_misery)
print()

Common movies:
shape: (7, 4)
┌───────┬────────────────────────────┬────────────┬─────────────────────┐
│ index ┆ title                      ┆ avg_rating ┆ least_misery_rating │
│ ---   ┆ ---                        ┆ ---        ┆ ---                 │
│ i32   ┆ str                        ┆ f32        ┆ f32                 │
╞═══════╪════════════════════════════╪════════════╪═════════════════════╡
│ 659   ┆ Godfather, The (1972)      ┆ 3.251274   ┆ 2.324125            │
│ 921   ┆ Blues Brothers, The (1980) ┆ 2.746333   ┆ 1.995953            │
│ 793   ┆ Die Hard (1988)            ┆ 2.75547    ┆ 1.924579            │
│ 2194  ┆ Body Heat (1981)           ┆ 2.795218   ┆ 1.875343            │
│ 901   ┆ Brazil (1985)              ┆ 2.66538    ┆ 1.766048            │
│ 474   ┆ Blade Runner (1982)        ┆ 2.523386   ┆ 1.755231            │
│ 3633  ┆ White Water Summer (1987)  ┆ 2.776587   ┆ 1.679273            │
└───────┴────────────────────────────┴────────────┴─────────────────────┘
Unique mo

From the previous results we can see that the top 10 movies are mostly same for the average and least misery aggregation methods. Both methods have 7 movies in common and 3 unique movies. The order of the common movies varies alot. This result is expected since the two methods are based on different principles.

From the predicted scores in both methods we can see that the predictions are not very good. This however, is not a fault of the measure but the group of users that we have chosen. The group of users do not seem to share a preference when it comes to movies. If we had chosen a group of users that have more similar taste, both methods would show better results. From the individual ratings of the users we can see that all users have movies that they would presumably like very much

The results we have now are both a tradeoff and will now leave at least one user disliking the movie the group would watch.

#### f)

Disagreement between the users in a group

We calculate disagreement by taking the max and min ratings of the group members for each movie. Min rating is reduced from max rating in order to have a disagreement score for the movie. These disagreement scores are then averaged in order to get the average disagreement score for predicted movies.  

Let's calculate disagreement for both average and least misery aggregated recommendations to see which one is better.

In [118]:
def get_average_group_disagreement(df):
    top_10_movie_indices = (
        df
        .select("index")
        .to_numpy()
        .reshape(-1)
        .tolist()
    )
    
    min_ratings = (
        predictions_df
        .select([str(x) for x in top_10_movie_indices])
        .select(pl.all().min())
        .to_numpy()
        .reshape(-1)
    )
    
    max_ratings = (
        predictions_df
        .select([str(x) for x in top_10_movie_indices])
        .select(pl.all().max())
        .to_numpy()
        .reshape(-1)    
    )
    
    return np.sum(max_ratings-min_ratings) / len(min_ratings)


print("Average group disagreement for least misery aggregation:")    
print(get_average_group_disagreement(top_10_movies_least_misery_df))
print()
print("Average group disagreement for average aggregation:")
print(get_average_group_disagreement(top_10_movies_avg_df))

Average group disagreement for least misery aggregation:
2.1357725

Average group disagreement for average aggregation:
2.171708


From the average group disagreement scores we can see how the least misery aggregation has lower disagreement score. It is thus more pleasing for all users in the group. We select least misery aggregated recommendations to be our final recommendation.